In [ ]:
import random
import time
import json
import threading

class Player:
    """Creates the class for the Player to make user inputs"""
    def __init__(self, name, job):
        """Initializes name, and job of player"""
        self.name = name
        self.job = job
        self.inventory = []
        self.skills = self.skills()
        self.health = self.set_health()
        self.mana = self.set_mana()
        self.starting_equipment()
        
    def set_health(self):
        if self.job == "Wizard":
            return 60
        elif self.job == "Warrior":
            return 100
        elif self.job == "Ranger":
            return 80
        
    def set_mana(self):
        if self.job == "Wizard":
            return 100
        elif self.job == "Warrior":
            return 50
        elif self.job == "Ranger":
            return 80

    def skills(self):
        if self.job == "Wizard":
            return {"Fireball": (30, 38, 0, 25), "Magic Missile": (20, 40, 0, 30)}
        elif self.job == "Warrior":
            return {"Slash": (25, 33, 0, 20), "Stab": (20, 25, 3, 38)}
        elif self.job == "Ranger":
            return {"Triple Shot": (10, 30, 0, 15), "Deadshot": (30, 40, 0, 35)}
    
    def starting_equipment(self):
        if self.job == "Wizard":
            self.inventory.extend([Weapons("Oak Staff", "Reliable staff and perfect for the new wizard.", 12),
                                   Armor("Cloth Robe", "Offers little protection. Looks cool though.", 10),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
        elif self.job == "Warrior":
            self.inventory.extend([Weapons("Iron Sword", "A well crafted iron sword.", 10),
                                   Armor("Iron Armor", "This should protect me.", 25),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
        elif self.job == "Ranger":
            self.inventory.extend([Weapons("Oak Longbow", "A beautiful longbow made of luxurious Oak..", 12),
                                   Armor("Ranger's Robe", "Standard cloak given to rangers starting their journey.", 8),
                                   HP_Potions("Lesser Health Potion", "Heals 10 health.", 10),
                                   MP_Potions("Lesser Mana Potion", "Recover 10 mana.", 10)])
       
    def status(self):
        print(f"Job: {self.job}")
        print(f"{self.name}'s HP: {self.health}")
        print(f"{self.name}'s MP: {self.mana}")
        print("Inventory: ")
        for i, item in enumerate(self.inventory, start = 1):
            print(f"{i}. {item.name} | Description: {item.description}")
            if isinstance(item, Weapons):
                print(f"Damage: {item.damage}")
            elif isinstance(item, HP_Potions):
                print(f"Heal Amount: {item.heal_amount}")
            elif isinstance(item, MP_Potions):
                print(f"Recover Amount: {item.rec_amount}")
            elif isinstance(item, Armor):
                print(f"Defense: {item.defense}")
        print ("Skills: ")
        for skill, (min_damage, max_damage, _, mana_cost) in self.skills.items():
            print("-", skill, "Damage range: ", min_damage, " - ", max_damage, "Mana cost:", mana_cost)
            
    def add_item(self, item):
        self.inventory.append(item)
        
    def take_damage(self, damage):
        self.health -= damage
        
    def use_mana(self, mana_cost):
        self.mana -= mana_cost
    
    def equip_gear(self):
        choice = input("Enter the what item you want to equip (or 0 to cancel): ").strip()
        if choice.isdigit():
            index = int(choice) - 1
            if 0 <= index < len(self.inventory):
                item = self.inventory[index]
                if isinstance(item, Weapons):
                    self.equipped_weapon = item
                    print(f"{item.name} has been equipped!")
                elif isinstance(item, Armor):
                    self.equipped_armor = item
                    print(f"{item.name} has been equipped!")
                else:
                    print("You cannot equip this item.")
            elif index == -1:
                print("You chose not to equip anything.")
            else:
                print("Invalid choice.")
        else:
            print("Invalid input. Please enter a valid number.")
        
    def unequip_gear(self, item_index):
        if item_index >= 0 and item_index < len(self.inventory):
            item = self.inventory.pop(item_index)
            print(f"{item.name} has been unequipped.")
            return item
        else:
            print("Invalid item index.")
            return None

In [ ]:
class Monster:
    """Creates class for monsters to fight player"""
    def __init__(self, kind, health_range, attack_range):
        """Initializes attributes of the monsters"""
        self.kind = kind
        self.health = random.randint(*health_range)
        self.attack = random.randint(*attack_range)
    
    def take_damage(self, damage):
        self.health -= damage

In [ ]:
def generate_monsters():
    monster_stats = {"Goblin": ([20, 45], [5, 12]), "Skeleton": ([30, 48], [10,15]), "Zombie": ([50,75], [15, 20])}
    kind = random.choice(list(monster_stats.keys()))
    health_range, attack_range = monster_stats[kind]
    return Monster(kind, health_range, attack_range) 

In [ ]:
class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

In [ ]:
class Weapons(Item):
    def __init__(self, name, description, damage):
        super().__init__(name, description)
        self.damage = damage

In [ ]:
class HP_Potions(Item):
    def __init__(self, name, description, heal_amount):
        super().__init__(name, description)
        self.heal_amount = heal_amount

In [ ]:
class MP_Potions(Item):
    def __init__(self, name, description, rec_amount):
        super().__init__(name, description)
        self.rec_amount = rec_amount

In [ ]:
class Armor(Item):
    def __init__(self, name, description, defense):
        super().__init__(name, description)
        self.defense = defense

In [ ]:
def save_game(player):
    data = {
        "name": player.name,
        "job": player.job,
        "inventory": [item.__dict__ for item in player.inventory],
        "health": player.health,
        "mana": player.mana
    }
    with open("save_game.json", "w") as file:
        json.dump(data, file, indent = 4)
    print("Game saved.")

In [ ]:
def load_game(weapons, hp_pot, mp_pot, armor):
    try:
        with open("save_game.json", "r") as file:
            data = json.load(file)
        player_name = data["name"]
        job = data["job"]
        player = Player(player_name, job)
        player.health = data["health"]
        player.mana = data["mana"]
        for item_name in data["inventory"]: #populates inventory of player
            item = find_item_name(item_name, weapons, hp_pot, mp_pot, armor)
            if item:
                player.add_item(item)
        print("Game loaded.")
        return player
    except FileNotFoundError:
        print("No previous save data found.")
        return None

In [ ]:
def find_item_name(item_name, weapons, hp_pot, mp_pot, armor):
    for item in weapons + hp_pot + mp_pot + armor:
        if item.name == item_name:
            return item
    return None

In [ ]:
def skill_choice(player):
    print("Choose a skill to use: ")
    for idx, (skill, damage_values) in enumerate(player.skills.items(), start = 1):
        if len(damage_values) == 4:
            min_damage, max_damage, duration, mana_cost = damage_values
            print(f"{idx}. {skill} Damage range: {min_damage} - {max_damage}, DoT Duration: {duration} Mana Cost: {mana_cost}")
        else:
            print(f"{idx}. {skill} Has invalid format")
    choice = input("Enter the skill of your choice: ").strip()
    skills = list(player.skills.keys())
    if choice.isdigit() and 1 <= int(choice) <= len(player.skills):
        return skills[int(choice) - 1]
    else:
        print("Invalid input. Choosing default skill.")
        return skills[0]

In [ ]:
dot_event = threading.Event()
def DoT(player, monster, damage, duration):
    print(f"The {monster.kind} is bleeding from the attack!")
    for _ in range(duration):
        dot_event.wait()
        dot_event.clear()
        time.sleep(1)
        monster.take_damage(damage)
        print(f"The {monster.kind} takes {damage} damage from bleed!")
        m_attack = random.randint(5, 50)
        player.take_damage(m_attack)
        print(f"The {monster.kind} attacked you for {m_attack} damage!")
        print(f"Your health is: {player.health}.")
        if player.health <= 0:
            game_over()
    print(f"The {monster.kind} has recovered from bleed.")

In [ ]:
def game_over():
    print("GAME OVER!")
    exit()

In [ ]:
def item_drop(weapons, hp_pot, mp_pot, armor):
    drop_chance = random.random()
    if drop_chance < 1:
        return random.choice(weapons + hp_pot + mp_pot + armor)
    return None

In [ ]:
def pickup(player, item):
    print(f"A {item.name} was dropped!")
    pick_up = input(f"Do you want to pick up the {item.name}? (yes/no)").lower()
    if pick_up == "yes":
        player.add_item(item)
        print(f"You put the {item.name} into your inventory.")
    else:
        print("You chose to leave the item.")

In [ ]:
def Encounter(player, monster, weapons, hp_pot, mp_pot, armor):
    print(f"You encoutered a {monster.kind}!")
    while player.health > 0 and monster.health > 0:
        p_action = input("Do you want to (A) Basic attack, (B) Use a skill, (C) Use an item, or (D) Run away? ").lower()
        if p_action == "a":
            if player.equipped_weapon:
                p_attack = random.randint(5, 15) + player.equipped_weapon.damage
            else:
                p_attack = random.randint(5, 15)
            monster.take_damage(p_attack)
            print(f"You hit the {monster.kind} for {p_attack} damage!")
            if monster.health <= 0:
                print(f"Congrats! You defeated the {monster.kind}!")
                if random.random() < 1:
                    item = item_drop(weapons, hp_pot, mp_pot, armor)
                    if item:
                        pickup(player, item)
                return True
            if player.equipped_armor:
                m_attack = max(0, random.randint(5,40) - player.equipped_armor.defense)
            else:
                m_attack = max(0, random.randint(5,40))
            player.take_damage(m_attack)
            print(f"The {monster.kind} attacked you for {m_attack} damage!")
            print(f"Your health is: {player.health}.")
        elif p_action == "b":
            skill_name = skill_choice(player)
            if skill_name in player.skills:
                min_damage, max_damage, duration, mana_cost = player.skills[skill_name]
                skill_damage = random.randint(min_damage, max_damage)
                if player.mana >= mana_cost:
                    player.use_mana(mana_cost)
                    if skill_name == "Stab":
                        dot_thread = threading.Thread(target=DoT, args = (player, monster, skill_damage, duration))
                        dot_thread.start()
                        if monster.health <=0:
                            print(f"You have slain the {monster.kind}!")
                    else:
                        monster.take_damage(skill_damage)
                        print(f"You used {skill_name} on the {monster.kind} for {skill_damage} damage!")
                    if monster.health <= 0:
                        print(f"You have slain the {monster.kind}!")
                        if random.random() < 0.5:
                            item = item_drop(weapons, hp_pot, mp_pot, armor)
                            if item:
                                pickup(player, item)
                        return True
                    if player.equipped_armor:
                        m_attack = max(0, random.randint(5,40) - player.equipped_armor.defense)
                    else:
                        m_attack = max(0, random.randint(5,40))
                    player.take_damage(m_attack)
                    print(f"The {monster.kind} attacked you for {m_attack} damage!")
                    print(f"Your health is: {player.health}.")
                    print(f"Your mana is: {player.mana}.")
                else:
                    print("You do not have enough mana to use this skill!")
        elif p_action == "c":
            if player.inventory:
                print("Inventory:")
                for i, item in enumerate(player.inventory, start = 1):
                    print(f"{i}. {item.name} - {item.description}")
                choice = input("Enter the number of the item you want to use: ")
                if choice.isdigit() and 1 <= int(choice) <=len(player.inventory):
                    chosen_item = player.inventory[int(choice) - 1]
                    if isinstance(chosen_item, HP_Potions):
                        player.health += chosen_item.heal_amount
                        if player.health > player.set_health():
                            player.health = player.set_health()
                        print(f"You restored {chosen_item.heal_amount} HP.")
                    elif isinstance(chosen_item, MP_Potions):
                        player.mana += chosen_item.rec_amount
                        if player.mana > player.set_mana():
                            player.mana = player.set_mana()
                        print(f"You recovered {chosen_item.rec_amount} MP.")
                    else:
                        print("You can't use that item in combat.")
                    player.inventory.remove(chosen_item)
                else:
                    print("Invalid choice. Please choose a valid number.")
            else:
                print("Your inventory is empty.")
        elif p_action == "d":
            print("You ran away!")
            return False
        else:
            print("Invalid input. Please enter a proper choice.")
        dot_event.set()
        if player.health <= 0:
            print("Game has ended. terminating program")
            game_over()
            return False
    return True

In [ ]:
def choose_job():
    print("Choose your job adventurer:" )
    print(" 1. Wizard\n", "2. Warrior\n", "3. Ranger\n")
    choice = input("Enter your choice: ").strip()
    if choice == "1":
        return "Wizard"
    elif choice == "2":
        return "Warrior"
    elif choice == "3":
        return "Ranger"
    else:
        print("Invalid input. Choosing default class: Wizard")
        return "Wizard"

In [ ]:
def load_assets(filename):
    with open(filename, "r") as file:
        data = json.load(file)
    weapons = [Weapons(item["Name"], item["Description"], item["Damage"]) for item in data["Weapons"]]
    armor = [Armor(item["Name"], item["Description"], item["Defense"]) for item in data["Armor"]]
    hp_pot = [HP_Potions(item["Name"], item["Description"], item["Heal_amount"]) for item in data["Health Potions"]]
    mp_pot = [MP_Potions(item["Name"], item["Description"], item["Rec_amount"]) for item in data["Mana Potions"]]
    return weapons, hp_pot, mp_pot, armor

In [ ]:
def main():
    weapons, hp_pot, mp_pot, armor = load_assets("assets.json") #loads items from assets json file. 
    player = load_game(weapons, hp_pot, mp_pot, armor)
    if player is None:
        player_name = input("What is your name adventurer? ")
        job = choose_job()
        player = Player(player_name, job)
    monsters_slain = 0
    while True:
        if monsters_slain == 3:
            print("Congratulations You have defeated three monsters and won the game!")
            break
        choice = input("Do you want to (1) continue exploring, (2) view status, (3) remove gear, (4) save progress, or (5) quit? ").strip()
        if choice == "1":
            if random.random() < 0.5:
                monster = generate_monsters()
                if Encounter(player, monster, weapons, hp_pot, mp_pot, armor):
                    monsters_slain += 1
            elif random.random() > 0.5:
                print("You continued moving with no issues.\nDid you want to (1) continue or (2) view status?")
        elif choice == "2":
            player.status()
            player.equip_gear()
        elif choice == "3":
            unequip_choice = input("Do you want to unequip an item? (yes/no): ").strip().lower()
            if unequip_choice == "yes":
                player.status()
                item_index = int(input("Enter the number associated with the item you want to unequip: ")) -1
                unequipped_item = player.unequip_item(item_index)
                print("Item unequipped.")
            else:
                print("No items were unequipped.")
        elif choice == "4":
            save_game(player)
        elif choice == '5':
            save_choice = input("Do you want to save before quitting? (yes/no)").strip().lower()
            if save_choice == "yes":
                save_game(player)
            print("Thanks for playing!")
            break
        else:
            print("Invalid input. Please enter a valid choice.")
        
if __name__ == "__main__":
    main()